- 层和块

nn.Flatten  在使用  nn.Sequential  时通常用于将输入张量的形状从多维转换为二维，以便可以传递给全连接层（ nn.Linear ）。在深度学习中，很多时候我们处理的输入数据是图像或其他多维结构，这些数据在经过卷积层（ nn.Conv2d ）后会保持多维形状。 
 



In [2]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
#使用nn.Sequential定义了一个module
X = torch.rand(2, 20)
net(X)


tensor([[ 0.0945, -0.0008, -0.1331, -0.1691,  0.0782,  0.1038,  0.0601,  0.0299,
         -0.0320,  0.0410],
        [ 0.1115, -0.0645, -0.1802, -0.0907,  0.0830, -0.0400, -0.0090,  0.0120,
          0.0404,  0.1634]], grad_fn=<AddmmBackward0>)

- 自定义块

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        #调用父类的构造函数，初始化nn.Module的部分。
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        #前向传播方法定义了模型如何处理输入数据以生成输出。 
        return self.out(F.relu(self.hidden(X)))


实例化MLP的层，然后在每次调用正向传播函数时调用这些层

In [4]:
net = MLP()
X = torch.rand(2, 20)
net(X)

tensor([[-0.3908,  0.1033,  0.1658,  0.1251,  0.0143, -0.1079, -0.1030,  0.1867,
          0.1416,  0.1509],
        [-0.4040, -0.0690,  0.1297,  0.2205, -0.0520, -0.0201, -0.1991,  0.3000,
          0.0880,  0.2470]], grad_fn=<AddmmBackward0>)

- 定义顺序块

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block
        
    def forward(self, X):
        for block in self._modules.values():
            print(block)
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)
net(X)


Linear(in_features=20, out_features=256, bias=True)
ReLU()
Linear(in_features=256, out_features=10, bias=True)


tensor([[ 0.1149,  0.1179, -0.1235,  0.0116,  0.0412,  0.2265,  0.0456,  0.0767,
         -0.0776,  0.0018],
        [ 0.0990,  0.1188,  0.0084,  0.1107,  0.0910,  0.1888, -0.0449,  0.1171,
         -0.1423,  0.0553]], grad_fn=<AddmmBackward0>)

- 自定义前向传播

In [6]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand(20, 20, requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        #F.relu()
        F.relu(X @ (self.rand_weight+1))
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
net = FixedHiddenMLP()
X = torch.rand(2, 20)
net(X)


tensor(-0.0429, grad_fn=<SumBackward0>)

- 混合组合块 套娃

In [7]:
# 理解：嵌套使用，因为现有的层和网络都是nn.Module的子类
# 混合代培各种组合块的方法
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64),nn.ReLU(),
                                nn.Linear(64,32),nn.ReLU())
        self.linear = nn.Linear(32,16)
        
    def forward(self, X):
        return self.linear(self.net(X))
    
chimear = nn.Sequential(NestMLP(),nn.Linear(16,20),FixedHiddenMLP())
X = torch.rand(2,20)
chimear(X)

tensor(0.1993, grad_fn=<SumBackward0>)